In [6]:
import torch;
import torch.nn as nn;
import torch.optim as optim;
import torch.nn.functional as F;
from torch.utils.data import DataLoader;
import torchvision.datasets as datasets;
import torchvision.transforms as transforms;

device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')

print(device, torch.__version__)

cpu 2.4.1


In [7]:
class CNN(nn.Module):
    def __init__(self, in_channels = 1, num_classes = 10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=1,
            out_channels=8,
            kernel_size=(3,3),
            stride=(1,1), 
            padding=(1,1)
        )
        self.pool = nn.MaxPool2d(
            kernel_size=(2,2), 
            stride=(2,2)
        )
        self.conv2 = nn.Conv2d(
            in_channels=8,
            out_channels=16,
            kernel_size=(3,3),
            stride=(1,1), 
            padding=(1,1)
        )
        self.fc1 = nn.Linear(16*7*7, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)

        return x
    

In [8]:
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 5

train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [9]:
model = CNN().to(device=device)
x = torch.randn(64,1,28,28)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device=device)
        targets = targets.to(device=device)

        print(epoch, batch_idx, data.shape, targets.shape)

        scores = model(data)
        loss = criterion(scores, targets)

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

0 0 torch.Size([64, 1, 28, 28]) torch.Size([64])
0 1 torch.Size([64, 1, 28, 28]) torch.Size([64])
0 2 torch.Size([64, 1, 28, 28]) torch.Size([64])
0 3 torch.Size([64, 1, 28, 28]) torch.Size([64])
0 4 torch.Size([64, 1, 28, 28]) torch.Size([64])
0 5 torch.Size([64, 1, 28, 28]) torch.Size([64])
0 6 torch.Size([64, 1, 28, 28]) torch.Size([64])
0 7 torch.Size([64, 1, 28, 28]) torch.Size([64])
0 8 torch.Size([64, 1, 28, 28]) torch.Size([64])
0 9 torch.Size([64, 1, 28, 28]) torch.Size([64])
0 10 torch.Size([64, 1, 28, 28]) torch.Size([64])
0 11 torch.Size([64, 1, 28, 28]) torch.Size([64])
0 12 torch.Size([64, 1, 28, 28]) torch.Size([64])
0 13 torch.Size([64, 1, 28, 28]) torch.Size([64])
0 14 torch.Size([64, 1, 28, 28]) torch.Size([64])
0 15 torch.Size([64, 1, 28, 28]) torch.Size([64])
0 16 torch.Size([64, 1, 28, 28]) torch.Size([64])
0 17 torch.Size([64, 1, 28, 28]) torch.Size([64])
0 18 torch.Size([64, 1, 28, 28]) torch.Size([64])
0 19 torch.Size([64, 1, 28, 28]) torch.Size([64])
0 20 torch

In [18]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0

    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)

            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(f'{num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

    model.train()

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

59097/60000 with accuracy 98.50
9822/10000 with accuracy 98.22
